- Goal of this tutorial is to showcase how to retrieve information from PDF papers, including both images and textual information.
- The extracted data will be stored to a vector database (ChromaDB) 
- The agent will provide literature informed answers to human questions.

To be able to run the tutorial you need to have and OPENAI API key

In [8]:
OPENAI_API_KEY = "" # Your OpenAI API key

We also need to download the pretrained weights of the YOLO model for image cropping

In [ ]:
# placeholder for downloading the checkpoints from google cloud

In [2]:
import numpy as np
import torch
print(f"NumPy version: {np.__version__}")
print(f"PyTorch version: {torch.__version__}")

NumPy version: 1.24.3
PyTorch version: 2.0.1+cpu


In [1]:
from matagen.agents import *
from matagen.scraping.pdf_scraper import pdf_scraper_tool
from matagen.config.settings import OPENAI_API_KEY

In [2]:
pdf_file_path = "DataMiningAgents/data/pdf_examples"
output_base_dir = "outputs/"
pdf_scraper_tool(pdf_file_path, output_base_dir, paper_name="pdf_papers", api_key = OPENAI_API_KEY)

self.results_directory_tool outputs\pdf_papers
self.results_directory_tool outputs\pdf_papers
self.results_directory_tool outputs\pdf_papers
Running PDF Scraper
articles []
>>> Time Elapsed: 0.00 sec (0 articles)
Running Caption Distributor
>>> Time Elapsed: 0.00 sec (0 captions)
Running Figure Separator
>>> Time Elapsed: 0.00 sec (0 figures)
Matching Image Objects to Caption Text
>>> SUCCESS! Matching objects from figure: page_9_img_1.png
key_value page_2_img_1.png {'title': 'Self‐Assembly Behavior of an Oligothiophene‐Based Conjugated Liquid Crystal and Its Implication for Ionic Conductivity Characteristics', 'article_name': 'C:/Users/kvriz/Desktop/DataMiningAgents/data/pdf_examples\\4T-PEO4', 'image_url': [], 'figure_name': 'page_2_img_1.png', 'full_caption': 'Scheme 1. Synthetic procedure of 4T/PEO4.', 'figure_path': 'output\\figures\\page_2_img_1.png', 'master_images': [{'classification': 'subfigure', 'confidence': 1.0, 'height': 782, 'width': 2333, 'geometry': [{'x': 0, 'y': 0}, 

{'page_2_img_1.png': {'title': 'Self‐Assembly Behavior of an Oligothiophene‐Based Conjugated Liquid Crystal and Its Implication for Ionic Conductivity Characteristics',
  'article_name': 'C:/Users/kvriz/Desktop/DataMiningAgents/data/pdf_examples\\4T-PEO4',
  'image_url': [],
  'figure_name': 'page_2_img_1.png',
  'full_caption': 'Scheme 1. Synthetic procedure of 4T/PEO4.',
  'figure_path': 'output\\figures\\page_2_img_1.png',
  'master_images': [{'classification': 'subfigure',
    'confidence': 1.0,
    'height': 782,
    'width': 2333,
    'geometry': [{'x': 0, 'y': 0},
     {'x': 2333, 'y': 0},
     {'x': 0, 'y': 782},
     {'x': 2333, 'y': 782}],
    'subfigure_label': {'text': '0',
     'geometry': [{'x': 0, 'y': 0},
      {'x': 2333, 'y': 0},
      {'x': 0, 'y': 782},
      {'x': 2333, 'y': 782}]},
    'caption': [],
    'keywords': []}],
  'article_url': [],
  'license': [],
  'open': [],
  'unassigned': {'master_images': [],
   'dependent_images': [],
   'inset_images': [],
   '

In [ ]:
# Create a tensor board to visualize the image-text pairs
data = r"outputs\pdf_papers\retrieved_image_caption_pairs.json"
import json
data_dict = json.load(open(data, "r"))

for i in data_dict["records"]:
    print(i['caption'])
    print(i['image'])
    print(i["caption_summary"])

Scheme 1. Synthetic procedure of 4T/PEO4.
outputs\pdf_papers\images_folder\page_2_img_1_0.jpg
POM image of 4T/PEO4-LiTFSI complex (r = 0.05) on cooling from isotropic melt at 97 and 50 °C (scale bar: 100 μm).
outputs\pdf_papers\images_folder\page_3_img_1_f.jpg
POM image of pristine 4T/PEO4 (r = 0) on cooling from isotropic melt at 95 and 60 °C.
outputs\pdf_papers\images_folder\page_3_img_1_b.jpg
POM image of 4T/PEO4-LiTFSI complex (r = 0.01) on cooling from isotropic melt at 75 and 40 °C.
outputs\pdf_papers\images_folder\page_3_img_1_d.jpg
First cooling scans (indicated by black arrows) and second subsequent heating scans (indicated by red arrows) of 4T/PEO-LiTFSI complexes (r = 0, r = 0.01, r = 0.05).
outputs\pdf_papers\images_folder\page_3_img_1_a.jpg
POM image of 4T/PEO4-LiTFSI complex (r = 0.05) on cooling from isotropic melt at 97 and 50 °C (scale bar: 100 μm).
outputs\pdf_papers\images_folder\page_3_img_1_g.jpg
POM image of pristine 4T/PEO4 (r = 0) on cooling from isotropic melt 

In [ ]:
from chromadb.utils import embedding_functions

embedding_functions.SentenceTransformerEmbeddingFunction("all-MiniLM-L6-v2")

RuntimeError: Failed to import transformers.modeling_utils because of the following error (look up to see its traceback):
module 'torch' has no attribute 'compiler'

In [1]:
# Use the multimodal information to create a RAG Agent
from matagen.analysis import multimodal_db
outputs_folder = "outputs/pdf_papers"
json_file_name = "retrieved_image_caption_pairs.json"  # Name of the JSON file inside OUTPUTS_FOLDER
text_folder_name = "pdf"      # Name of the subfolder with the retrieved text from the PDF file
chroma_db_path = "./my_multimodal_chroma_db" # Directory to store the persistent ChromaDB database
collection_name = "multimodal_rag_collection" # Name for the ChromaDB collection
text_embedding_model = "all-MiniLM-L6-v2" # Standard good default

multimodal_db.create_multimodal_chromadb(outputs_folder, json_file_name, text_folder_name, chroma_db_path, collection_name, text_embedding_model)

Starting ChromaDB Population (Text Chunks + Image Captions)
Script directory: c:\Users\kvriz\Desktop\DataMiningAgents\notebooks
Resolved paths:
  Outputs folder: outputs/pdf_papers
  JSON file path: outputs/pdf_papers\retrieved_image_caption_pairs.json
  Text folder path: outputs/pdf_papers\pdf
  ChromaDB storage path: c:\Users\kvriz\Desktop\DataMiningAgents\notebooks\./my_multimodal_chroma_db

--- Loading Source Data ---
Successfully loaded 30 image records from outputs/pdf_papers\retrieved_image_caption_pairs.json
Finished finding 1 text files.
Finished chunking 1 text files.

--- Preparing Documents from Image Captions (30 records) ---
Finished preparing 30 documents from image captions.

--- Initializing ChromaDB ---
Using persistent storage at: c:\Users\kvriz\Desktop\DataMiningAgents\notebooks\./my_multimodal_chroma_db

Setting up TEXT embedding function: all-MiniLM-L6-v2


c:\Users\kvriz\miniconda3\envs\mmatagen\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Error loading text embedding function model 'all-MiniLM-L6-v2': Failed to import transformers.modeling_utils because of the following error (look up to see its traceback):
module 'torch' has no attribute 'compiler'
Will attempt to use ChromaDB default.

Accessing collection: 'multimodal_rag_collection'
DEBUG: Embedding function being used: None
Collection 'multimodal_rag_collection' ready (using text embeddings).

--- Adding Data to ChromaDB Collection ---

Preparing 99 total text documents (chunks + image captions) for batch addition...
Adding data in 1 batches of size 500...
  Adding batch 1/1 (99 items)...
  Error during batch 1 upsert: You must provide an embedding function to compute embeddings.https://docs.trychroma.com/guides/embeddings in upsert.

Finished adding/updating 99 text documents.

--- Script Finished ---
Collection 'multimodal_rag_collection' now contains 0 items.
Total execution time: 8.42 seconds
ChromaDB data is stored in: c:\Users\kvriz\Desktop\DataMiningAgents\n

In [ ]:
# Query the multimodal database
from matagen.agents.multimodal_rag_agent import MultimodalAgent